<a href="https://colab.research.google.com/github/shivcoderss/RAG_chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pandas faiss-cpu sentence-transformers transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 23.4 MB/s eta 0:00:00


In [28]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import pipeline, AutoTokenizer

In [4]:
from google.colab import files
uploaded = files.upload()

Saving Training Dataset.csv to Training Dataset.csv


In [14]:
df = pd.read_csv("Training Dataset.csv")
df.fillna("N/A", inplace=True)
df_clean = df.replace("N/A", "")
docs = [" | ".join(f"{col}: {row[col]}" for col in df_clean.columns if str(row[col]).strip()) for _, row in df_clean.iterrows()]
print(f"Loaded {len(docs)} documents.")

Loaded 614 documents.


/tmp/ipython-input-14-1035686575.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'N/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("N/A", inplace=True)


In [22]:
embed_model = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = embed_model.encode(docs, show_progress_bar=True)

Batches:   0%|          | 0/20 [00:00<?, ?it/s]

In [23]:
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(np.array(doc_embeddings))
print("FAISS index created and populated.")

FAISS index created and populated.


In [24]:
def retrieve(query, k=5):
    query_emb = embed_model.encode([query])
    distances, indices = index.search(np.array(query_emb), k)
    return [docs[i] for i in indices[0]]

In [29]:
model_id = "google/flan-t5-small"
qa_pipeline = pipeline("text2text-generation", model=model_id, tokenizer=model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Device set to use cpu


In [30]:
def truncate_context(context, query, max_tokens=512):
    prompt_template = "Context: {}\n\nQuestion: {}\nAnswer:"
    context_parts = context.split('\n')
    while True:
        prompt = prompt_template.format("\n".join(context_parts), query)
        tokenized = tokenizer(prompt, return_tensors="pt")
        if tokenized.input_ids.shape[1] <= max_tokens:
            return "\n".join(context_parts)
        context_parts = context_parts[:-1]

In [31]:
def generate_answer(query, context):
    context = truncate_context(context, query)
    prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"
    result = qa_pipeline(prompt, max_new_tokens=100, do_sample=False)
    return result[0]['generated_text']

In [32]:
def rag_chatbot(query):
    retrieved_docs = retrieve(query)
    context = "\n".join(retrieved_docs)
    return generate_answer(query, context)

In [33]:
query = "What factors increase the chances of loan approval?"
response = rag_chatbot(query)
print("\nQuestion:", query)
print("\nAnswer:", response)

Token indices sequence length is longer than the specified maximum sequence length for this model (521 > 512). Running this sequence through the model will result in indexing errors



Question: What factors increase the chances of loan approval?

Answer: Loan ID is LP001267. The Gender is Female and Dependents are 2 and Education is Not Graduate. The CoapplicantIncome is 1881.0. The Loan amount is 167.0. The CoapplicantIncome is 1881.0. The CoapplicantIncome is 1881.0. The Loan amount is 167.0.
